
**INTALAR PYSPARK, IMPORTAR BIBLIOTECAS E ACESSO DO ARQUIVO NA BUCKET**

In [ ]:
pip install pyspark


     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=cbe3bec383a6bcf6cdd28e54a1a63f05827b750017e4ba456c807f52cdc98f1c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
#Autenticação do GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
#Selecionar o projeto
project_id = 'projeto-final-grupo03'
!gcloud config set project projeto-final-grupo03

Updated property [core/project].


In [ ]:
#Download do arquivo de um bucket no Google Cloud Storage
!gsutil cp gs://notebooks_pandas_gp03/acidentes_tratado.csv /tmp/acidentes_tratado.csv


Copying gs://notebooks_pandas_gp03/acidentes_tratado.csv...
| [1 files][ 45.8 MiB/ 45.8 MiB]                                                
Operation completed over 1 objects/45.8 MiB.                                     


In [ ]:
#Print do resultado para garantir que a transferencia funcionou.
#!cat /tmp/acidentes_tratado.csv

In [ ]:
#IMPORTAR BIBLIOTECAS
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql import SparkSession


In [ ]:
#INICIAR SESSÃO DO PYSPARK
spark = (SparkSession.builder
        .master("local")
        .appName("dataframe_projeto")
        .config("spark.ui.port', '4050") 
        .getOrCreate())

In [ ]:
spark

EXTRAÇÃO DOS DADOS

In [ ]:
#IMPORTAR DATAFRAME A PARTIR DO ARQUIVO CSV ANTERIORMENTE NORMALIZADO DO PANDAS
df = (spark.read.format('csv')
      .option("inferSchema", True)
      .option("header", True)
      .option('sep', ',')
      .load('/tmp/acidentes_tratado.csv')

)


In [ ]:
#MOSTRA O ESQUEMA DO DATAFRAME

df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id_acidente: integer (nullable = true)
 |-- id_pessoa: integer (nullable = true)
 |-- data: string (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- rodovia_br: integer (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_envolvido: string (nullable = true)
 |-- estado_fisico: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)



**ALTERAÇÔES PYSPARK**

In [ ]:
# MUDAR O TIPO DE DATA PARA DATE COM  CAST

df = df.withColumn("data", df.data.cast("date"))

In [ ]:
#MOSTRA O DATAFRAME
df.show()


+---+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+
|_c0|id_acidente|id_pessoa|      data|  dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|         sexo|
+---+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+
|  0|     182256|   403856|2019-01-01| terça-feira|04:00:00| CE|       116|    Animais na Pista|           Sem Vítimas|                 Vento|      Condutor|        Ileso|   35|    Masculino|
|  1|     182263|   402859|2019-01-01| terça-feira|05:00:00| MT|       158|Defeito Mecânico ...|           Sem Vítimas|        Garoa/Chuvisco|      Condutor|        Ileso|   30|    Masculino|
|  2|     182277|   402850|2019-01-01| t

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id_acidente: integer (nullable = true)
 |-- id_pessoa: integer (nullable = true)
 |-- data: date (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- rodovia_br: integer (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_envolvido: string (nullable = true)
 |-- estado_fisico: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)



In [ ]:
#CRIA NOVA COLUNA COM CLASSIFICAÇÃO DE REGIÃO DO BRASIL 

df1 = (df.withColumn('regiao', 
                     F.when((df.uf == 'PR') | (df.uf == 'RS') | (df.uf == 'SC'), 'Sul')
                      .when((df.uf == 'SP') | (df.uf == 'RJ') | (df.uf == 'MG') | (df.uf == 'ES'), 'Sudeste')
                      .when((df.uf == 'MT') | (df.uf == 'DF') | (df.uf == 'GO') | (df.uf == 'MS'), 'Centro-Oeste')
                      .when((df.uf == 'RR') | (df.uf == 'AP') | (df.uf == 'AM') | (df.uf == 'PA') | (df.uf == 'AC') 
                      | (df.uf == 'RO') | (df.uf == 'TO'), 'Norte').otherwise('Nordeste')))


In [ ]:
df1.show()

+---+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+
|_c0|id_acidente|id_pessoa|      data|  dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|         sexo|      regiao|
+---+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+
|  0|     182256|   403856|2019-01-01| terça-feira|04:00:00| CE|       116|    Animais na Pista|           Sem Vítimas|                 Vento|      Condutor|        Ileso|   35|    Masculino|    Nordeste|
|  1|     182263|   402859|2019-01-01| terça-feira|05:00:00| MT|       158|Defeito Mecânico ...|           Sem Vítimas|        Garoa/Chuvisco|      Condutor|        Ileso|   30|   

In [ ]:
#DROPANDO INDEX

df1 = df1.drop('_c0')

In [ ]:
df1.show()

+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+
|id_acidente|id_pessoa|      data|  dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|         sexo|      regiao|
+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+
|     182256|   403856|2019-01-01| terça-feira|04:00:00| CE|       116|    Animais na Pista|           Sem Vítimas|                 Vento|      Condutor|        Ileso|   35|    Masculino|    Nordeste|
|     182263|   402859|2019-01-01| terça-feira|05:00:00| MT|       158|Defeito Mecânico ...|           Sem Vítimas|        Garoa/Chuvisco|      Condutor|        Ileso|   30|    Masculino|Centro-Oe

In [ ]:
#CRIA NOVA COLUNA COM MÊS

df2 = df1.withColumn('mes', F.month(df1.data))


In [ ]:
#CRIA NOVA COLUNA COM ANO

df2 = df2.withColumn('ano', F.year(df2.data))

In [ ]:
df2.show()

+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+---+----+
|id_acidente|id_pessoa|      data|  dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|         sexo|      regiao|mes| ano|
+-----------+---------+----------+------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+---+----+
|     182256|   403856|2019-01-01| terça-feira|04:00:00| CE|       116|    Animais na Pista|           Sem Vítimas|                 Vento|      Condutor|        Ileso|   35|    Masculino|    Nordeste|  1|2019|
|     182263|   402859|2019-01-01| terça-feira|05:00:00| MT|       158|Defeito Mecânico ...|           Sem Vítimas|        Garoa/Chuvisco|      Condutor|       

In [ ]:
# FILTRO DATA INÍCIO PANDEMIA

filtroData = df3.filter((df3["data"] > "2020-03-01") & (df3["data"] < "2020-03-30")).show(10)

+-----------+---------+----------+-------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+---+----+
|id_acidente|id_pessoa|      data|   dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|         sexo|      regiao|mes| ano|
+-----------+---------+----------+-------------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+-------------+------------+---+----+
|     272733|   609933|2020-03-02|segunda-feira|01:00:00| MA|       316|Ultrapassagem Ind...|    Com Vítimas Fatais|             Céu Claro|      Condutor|        Ileso|   46|    Masculino|    Nordeste|  3|2020|
|     272746|   609473|2020-03-02|segunda-feira|04:30:00| SE|       101|Não Guardar Distâ...|   Com Vítimas Feridas|             Céu Claro|      Condutor| L

In [ ]:
#RETORNA O NÚMERO DE ACIDENTES DISTINTOS

df3.select(F.countDistinct(F.col('id_acidente')).alias('acidentes')).show(truncate=False)


+---------+
|acidentes|
+---------+
|130723   |
+---------+



In [ ]:
#MÉDIA DE IDADE GERAL

df3.select(F.round(F.avg('idade'),2).alias('media_idade_geral')).show()


+-----------------+
|media_idade_geral|
+-----------------+
|            39.51|
+-----------------+



In [ ]:
#MÉDIA DE IDADE POR ESTADO

df3.groupBy(F.col('uf')).avg('idade').select(F.col('uf'), F.round(F.col('avg(idade)'), 2).alias('media_idade')).show()


+---+-----------+
| uf|media_idade|
+---+-----------+
| SC|      38.01|
| RO|      39.18|
| PI|      38.48|
| AM|      37.38|
| RR|      39.46|
| GO|      38.75|
| TO|      37.37|
| MT|      39.26|
| SP|      41.15|
| ES|      38.79|
| PB|      39.49|
| RS|      39.45|
| MS|       38.9|
| AL|      56.81|
| MG|      40.02|
| PA|      39.02|
| BA|      40.29|
| SE|      38.72|
| PE|       40.0|
| CE|      38.94|
+---+-----------+
only showing top 20 rows



**CARREGAMENTO PARA CSV**

In [ ]:
#20 - SALVANDO O ARQUIVO NORMALIZADO NO BUCKET

.(df3.write.format("csv").option("header", "true")
                        .option("inferschema", "true")
                        .option("delimiter", ",")
                        .save("acidentes_pyspark"))
!gsutil cp -r acidentes_pyspark gs://notebooks_pyspark_sql_gp03

Copying file://acidentes_pyspark/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://acidentes_pyspark/part-00000-fac475b2-d2e8-48bb-835e-414b210612cc-c000.csv [Content-Type=text/csv]...
Copying file://acidentes_pyspark/.part-00000-fac475b2-d2e8-48bb-835e-414b210612cc-c000.csv.crc [Content-Type=application/octet-stream]...
Copying file://acidentes_pyspark/_SUCCESS [Content-Type=application/octet-stream]...
- [4 files][ 48.7 MiB/ 48.7 MiB]                                                
Operation completed over 4 objects/48.7 MiB.                                     


**CONSULTAS ESPECÍFICAS COM SPARKSQL**

In [ ]:
# CARREGA ARQUIVO
!gsutil cp gs://notebooks_pyspark_sql_gp03/acidentes_pyspark/part-00000-fac475b2-d2e8-48bb-835e-414b210612cc-c000.csv /tmp/acidentes_pyspark.csv

df = spark.read.csv('/tmp/acidentes_pyspark.csv', header=True, sep=",")
# REGISTRA UMA TABELA TEMPORÁRIA
df.createOrReplaceTempView("temp")
# SELECIONA TODOS OS DADOS DA TABELA TEMPORÁRIA
spark.sql("select * from temp limit 5").show()
# Select count of data in table
spark.sql("select count(*) as total_count from temp").show()


Copying gs://notebooks_pyspark_sql_gp03/acidentes_pyspark/part-00000-fac475b2-d2e8-48bb-835e-414b210612cc-c000.csv...
/
Operation completed over 1 objects/48.4 MiB.                                     
+-----------+---------+----------+-----------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+---------+------------+---+----+
|id_acidente|id_pessoa|      data| dia_semana| horario| uf|rodovia_br|      causa_acidente|classificacao_acidente|condicao_metereologica|tipo_envolvido|estado_fisico|idade|     sexo|      regiao|mes| ano|
+-----------+---------+----------+-----------+--------+---+----------+--------------------+----------------------+----------------------+--------------+-------------+-----+---------+------------+---+----+
|     182256|   403856|2019-01-01|terça-feira|04:00:00| CE|       116|    Animais na Pista|           Sem Vítimas|                 Vento|      Condutor|        Ileso|   35|Masculino| 

In [ ]:
# TOTAL DE ACIDENTES POR ANO

spark.sql("SELECT COUNT(DISTINCT id_acidente) AS total_2019 FROM temp WHERE ano = 2019").show()
spark.sql("SELECT COUNT(DISTINCT id_acidente) AS total_2020 FROM temp WHERE ano = 2020").show()

+----------+
|total_2019|
+----------+
|     67351|
+----------+

+----------+
|total_2020|
+----------+
|     63372|
+----------+



In [ ]:
# CONTA NÚMERO DE ACIDENTES POR MÊS DE JANEIRO A SETEMBRO EM 2019 E 2020


spark.sql("SELECT mes, COUNT(DISTINCT id_acidente) AS total_2019 FROM temp WHERE ano = 2019 AND mes BETWEEN 1 and 9 GROUP BY mes ORDER BY mes").show()
spark.sql("SELECT mes, COUNT(DISTINCT id_acidente) AS total_2020 FROM temp WHERE ano = 2020 AND mes BETWEEN 1 and 9 GROUP BY mes ORDER BY mes").show()

+---+----------+
|mes|total_2019|
+---+----------+
|  1|      5114|
|  2|      4959|
|  3|      5590|
|  4|      5418|
|  5|      5459|
|  6|      5572|
|  7|      5786|
|  8|      5698|
|  9|      5863|
+---+----------+

+---+----------+
|mes|total_2020|
+---+----------+
|  1|      5483|
|  2|      5461|
|  3|      4783|
|  4|      3873|
|  5|      4641|
|  6|      4720|
|  7|      5111|
|  8|      5622|
|  9|      5670|
+---+----------+

